# 合并数据框

Combining DataFrames

## 垂直组合

In [ ]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.width', 58)
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.0f}'.format

# 加载喀麦隆和波兰的数据
ltcameroon = pd.read_csv("data/ltcountry/ltcameroon.csv")
ltoman = pd.read_csv("data/ltcountry/ltoman.csv")

# 连接喀麦隆和波兰的数据
ltcameroon.shape
ltoman.shape
ltcameroon.columns
ltoman.columns
ltcameroon.columns.\
  symmetric_difference(ltoman.columns)
ltall = pd.concat([ltcameroon, ltoman])
ltall.country.value_counts()
ltall[['country','station','temperature',
  'latitude','latabs']].\
  sample(5, random_state=3)
ltall.groupby(['country'])['latabs'].count()

# 创建一个连接文件的函数
def concatfiles(filelist):
  directory = "data/ltcountry/"
  ltall = pd.DataFrame()
  for filename in filelist:
    ltnew = pd.read_csv(directory + filename + ".csv")
    print(filename + " has " + 
      str(ltnew.shape[0]) + " rows.")
    ltall = pd.concat([ltall, ltnew])
  return ltall

ltall = concatfiles(['ltcameroon','ltoman'])
ltall.country.value_counts()

# 连接所有数据文件
def concatallfiles():
  directory = "data/ltcountry"
  ltall = pd.DataFrame()
  for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
      fileloc = os.path.join(directory, filename)

      # 打开下一个文件
      with open(fileloc):
        ltnew = pd.read_csv(fileloc)
        print(filename + " has " + 
          str(ltnew.shape[0]) + " rows.")
        ltall = pd.concat([ltall, ltnew])

        # 检查列中的差异
        columndiff = ltall.columns.\
          symmetric_difference(ltnew.columns)
        if (not columndiff.empty):
          print("", "Different column names for:",
           filename, columndiff, "", sep="\n")
          
  return ltall

ltall = concatallfiles()

ltall[['country','station','month',
 'temperature','latitude']].\
 sample(5, random_state=1)

# 检查连接数据中的值
ltall.country.value_counts().sort_index()
ltall.groupby(['country']).\
  agg({'temperature':['mean','max','count'],
  'latabs':['mean','max','count']})

# 修复缺失值
ltall['latabs'] = np.where(ltall.country=="Oman", ltall.latitude, ltall.latabs)
ltall.groupby(['country']).\
  agg({'temperature':['mean','max','count'],
  'latabs':['mean','max','count']})


## 一对一合并

In [ ]:
import pandas as pd
pd.set_option('display.width', 52)
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:,.0f}'.format
nls97 = pd.read_csv("data/nls97f.csv", low_memory=False)
nls97.set_index("personid", inplace=True)
nls97add = pd.read_csv("data/nls97add.csv")

# 查看一些 NLS 数据
nls97.head()

nls97.shape
nls97add.head()
nls97add.shape

# 检查唯一 ID
nls97.originalid.nunique()==nls97.shape[0]
nls97add.originalid.nunique()==nls97add.shape[0]

# 创建一些不匹配的 ID
nls97 = nls97.sort_values('originalid')
nls97add = nls97add.sort_values('originalid')
nls97.loc[[135335,999406], "originalid"] = \
  nls97.originalid+10000
nls97.originalid.head(2)
nls97add.loc[[0,1], "originalid"] = \
  nls97add.originalid+20000
nls97add.originalid.head(2)
nls97.set_index("originalid", inplace=True)
nls97add.set_index("originalid", inplace=True)

# 使用连接进行左连接
nlsnew = nls97.join(nls97add, how="left", on=None, validate="many_to_many")
nlsnew.loc[nlsnew.index>9999, ['gender','birthyear','motherage','parentincome']]

# 进行左连接和合并
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how="left", validate="many_to_many")
nlsnew.loc[nlsnew.index>9999, ['gender','birthyear','motherage','parentincome']]

# 进行右连接
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how="right", validate="many_to_many")
nlsnew.loc[nlsnew.index>9999, ['gender','birthyear','motherage','parentincome']]

# 进行内连接
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how="inner", validate="many_to_many")
nlsnew.loc[nlsnew.index>9999, ['gender','birthyear','motherage','parentincome']]

# 做外连接
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how="outer", validate="many_to_many")
nlsnew.loc[nlsnew.index>9999, ['gender','birthyear','motherage','parentincome']]

# 创建一个检查 ID 不匹配的函数
def checkmerge(dfleft, dfright, idvar):
  dfleft['inleft'] = "Y"
  dfright['inright'] = "Y"
  dfboth = pd.merge(dfleft[[idvar,'inleft']],\
    dfright[[idvar,'inright']], on=[idvar], how="outer", validate="many_to_many")
  dfboth.fillna('N', inplace=True)
  print(pd.crosstab(dfboth.inleft, dfboth.inright))

checkmerge(nls97.reset_index(),nls97add.reset_index(), "originalid")

nlsnew = pd.merge(nls97, nls97add, left_on=['originalid'], right_on=['originalid'], how="right", validate="many_to_many")
nlsnew.loc[nlsnew.index>9999, ['gender','birthyear','motherage','parentincome']]

## 多列

In [ ]:
import pandas as pd
pd.set_option('display.width', 68)
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:,.0f}'.format
nls97weeksworked = pd.read_csv("data/nls97weeksworked.csv")
nls97colenr = pd.read_csv("data/nls97colenr.csv")

# 查看一些 NLS 数据
nls97weeksworked.loc[nls97weeksworked.\
  originalid.isin([2,3])]
    
nls97weeksworked.shape

nls97weeksworked.originalid.nunique()
nls97colenr.loc[nls97colenr.\
  originalid.isin([2,3])]

nls97colenr.shape
nls97colenr.originalid.nunique()

# 检查唯一 ID
nls97weeksworked.groupby(['originalid','year'])\
  ['originalid'].count().shape
nls97colenr.groupby(['originalid','year'])\
  ['originalid'].count().shape

# 创建一个检查 ID 不匹配的函数
def checkmerge(dfleft, dfright, idvar):
  dfleft['inleft'] = "Y"
  dfright['inright'] = "Y"
  dfboth = pd.merge(dfleft[idvar + ['inleft']],\
    dfright[idvar + ['inright']], on=idvar, how="outer", validate="many_to_many")
  dfboth.fillna('N', inplace=True)
  print(pd.crosstab(dfboth.inleft, dfboth.inright))

checkmerge(nls97weeksworked.copy(),nls97colenr.copy(), ['originalid','year'])

# 使用多个合并列
nls97workschool = \
  pd.merge(nls97weeksworked, nls97colenr,
  on=['originalid','year'], how="inner", validate="many_to_many")
nls97workschool.shape
nls97workschool.loc[nls97workschool.\
  originalid.isin([2,3])]


## 一对多合并

In [ ]:
import pandas as pd
pd.set_option('display.width', 53)
pd.set_option('display.max_columns', 5)
countries = pd.read_csv("data/ltcountries.csv")
locations = pd.read_csv("data/ltlocations.csv")

# 为地点和国家数据设置索引并打印几行
countries.set_index(['countryid'], inplace=True)
locations.set_index(['countryid'], inplace=True)
countries.head()
countries.index.nunique()==countries.shape[0]

locations[['locationid','latitude','stnelev']].head(10)

# 将国家和地点进行左连接
stations = countries.join(locations, how="left", on=None, validate="many_to_many")
stations[['locationid','latitude',
  'stnelev','country']].head(10)

# 重新加载地点文件并检查合并情况
countries = pd.read_csv("data/ltcountries.csv")
locations = pd.read_csv("data/ltlocations.csv")
def checkmerge(dfleft, dfright, idvar):
  dfleft['inleft'] = "Y"
  dfright['inright'] = "Y"
  dfboth = pd.merge(dfleft[[idvar,'inleft']],\
    dfright[[idvar,'inright']], on=[idvar], how="outer", validate="many_to_many")
  dfboth.fillna('N', inplace=True)
  print(pd.crosstab(dfboth.inleft, dfboth.inright))
  print(dfboth.loc[(dfboth.inleft=='N') | (dfboth.inright=='N')])

checkmerge(countries.copy(), locations.copy(), "countryid")

# 显示一个文件中的行，而不显示另一个文件中的行
countries.loc[countries.countryid.isin(["LQ","ST"])]
locations.loc[locations.countryid=="FO"]

# 合并地点和国家数据
stations = pd.merge(countries, locations, on=["countryid"], how="left", validate="many_to_many")
stations[['locationid','latitude',
  'stnelev','country']].head(10)
stations.shape
stations.loc[stations.countryid.isin(["LQ","ST"])].isnull().sum()

## 多对多合并

In [ ]:
import pandas as pd
pd.set_option('display.width', 64)
pd.set_option('display.max_columns', 6)
cmacitations = pd.read_csv("data/cmacitations.csv")
cmacreators = pd.read_csv("data/cmacreators.csv")

# 看看引文数据
cmacitations['citation'] = cmacitations.citation.str[0:15]
cmacitations.head(10)
cmacitations.shape
cmacitations.itemid.nunique()

# 查看创作者数据
cmacreators['creator'] = cmacreators.creator.str[0:15]
cmacreators.loc[:,['itemid','creator','birth_year',
 'creatorid']].head(10)
cmacreators.shape
cmacreators.itemid.nunique()
cmacreators.creatorid.nunique()

# 显示重复的引文合并值
cmacitations.itemid.value_counts().head(10)

# s显示创作者的重复合并值
cmacreators.itemid.value_counts().head(10)

# 检查合并
def checkmerge(dfleft, dfright, idvar):
  dfleft['inleft'] = "Y"
  dfright['inright'] = "Y"
  dfboth = pd.merge(dfleft[[idvar,'inleft']],\
    dfright[[idvar,'inright']], on=[idvar], how="outer", validate="many_to_many")
  dfboth.fillna('N', inplace=True)
  print(pd.crosstab(dfboth.inleft, dfboth.inright))

checkmerge(cmacitations.copy(), cmacreators.copy(), "itemid")

# 显示两个数据帧中重复的合并列
cmacitations.loc[cmacitations.itemid==124733]
cmacreators.loc[cmacreators.itemid==124733,
  ['itemid','creator','birth_year','title']]

# 进行多对多合并
cma = pd.merge(cmacitations, cmacreators, on=['itemid'], how="outer", validate="many_to_many")
cma.set_index("itemid", inplace=True)
cma.loc[124733, ['citation','creator','birth_year']]

cma.info()

cma.loc[75551]

## 合并程序

In [7]:
import pandas as pd
countries = pd.read_csv("data/ltcountries.csv")
locations = pd.read_csv("data/ltlocations.csv")

# 检查合并列是否匹配
def checkmerge(dfleft, dfright, mergebyleft, mergebyright):
  dfleft['inleft'] = "Y"
  dfright['inright'] = "Y"
  dfboth = \
    pd.merge(dfleft[[mergebyleft,'inleft']],\
    dfright[[mergebyright,'inright']],\
    left_on=[mergebyleft],\
    right_on=[mergebyright], how="outer", validate="many_to_many")
  dfboth.fillna('N', inplace=True)
  print(pd.crosstab(dfboth.inleft,
    dfboth.inright))
  print(dfboth.loc[(dfboth.inleft=='N') | \
    (dfboth.inright=='N')].head(20))

checkmerge(countries.copy(), locations.copy(), "countryid", "countryid")

# 合并地点和国家数据
stations = pd.merge(countries, locations, left_on=["countryid"], right_on=["countryid"],\
    how="left", validate="many_to_many")
stations[['locationid','latitude',
  'stnelev','country']].head(10)
stations.shape

inright  N      Y
inleft           
N        0      1
Y        2  27472
      countryid inleft inright
7363         FO      N       Y
9716         LQ      Y       N
13104        ST      Y       N


(27474, 7)